# Imports

In [1]:
import deeplake
import cv2
import os
import xml.etree.ElementTree as ET
import struct
from PIL import Image
import re

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from collections import defaultdict

int_inf = 9223372036854775807

/home/beav3r/Bachelor_work/bachelor_env/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.9.22) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [49]:
ds_downloaded = deeplake.load("hub://activeloop/kuzushiji-kanji")

Opening dataset in read-only mode as you don't have write permissions.


This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/kuzushiji-kanji

hub://activeloop/kuzushiji-kanji loaded successfully.



In [50]:
type(ds_downloaded)

deeplake.core.dataset.deeplake_cloud_dataset.DeepLakeCloudDataset

In [51]:
ds_downloaded.visualize()

In [52]:
ds_downloaded[0]

Dataset(path='hub://activeloop/kuzushiji-kanji', read_only=True, index=Index([0]), tensors=['images', 'labels'])

In [53]:
img0 = cv2.imread("/home/beav3r/Bachelor_work/Bachelor_work/data/追加/準備ができていない/Unicode/0.png", cv2.IMREAD_GRAYSCALE)
img1 = cv2.imread("/home/beav3r/Bachelor_work/Bachelor_work/data/追加/準備ができていない/Unicode/1.png", cv2.IMREAD_GRAYSCALE)
img_kkanji = cv2.imread("/home/beav3r/Bachelor_work/Bachelor_work/data/kkanji/kkanji2/U+4C61/e6eeb23552e1a21c.png", cv2.IMREAD_GRAYSCALE)
img0 = cv2.resize(img0, (64, 64))
img1 = cv2.resize(img1, (64, 64))
img0 = img0 / 255
img1 = img1 / 255

# Morphology opening and closing
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
img_kkanji_opened = cv2.morphologyEx(img_kkanji, cv2.MORPH_OPEN, kernel)
img_kkanji_closed = cv2.morphologyEx(img_kkanji_opened, cv2.MORPH_CLOSE, kernel)

# Save the thresholded image
cv2.imwrite("/home/beav3r/Bachelor_work/Bachelor_work/data/追加/準備ができていない/Unicode/thresholdedTest.png", img_kkanji_closed)

img_kkanji_closed.shape

(64, 64)

In [54]:
# Count amount of files in subsets and write into txt file

path = "/home/beav3r/Bachelor_work/Bachelor_work/data/kkanji/kkanji2"

def count_files(path):
    for root, dirs, files in os.walk(path):
        for dir in dirs:
            #Save to txt file
            with open("/home/beav3r/Bachelor_work/Bachelor_work/data/kkanji_classes_distribution.txt", "a") as file:
                file.write(dir + " " + str(len(os.listdir(os.path.join(root, dir)))) + "\n")
        
count_files(path)



In [56]:
"""Done"""
# Sort the classes by amount of files

# with open("/home/beav3r/Bachelor_work/Bachelor_work/kkanji_classes_distribution.txt", "r") as file:
#     lines = file.readlines()
#     lines.sort(key=lambda x: int(x.split()[1]), reverse=True)
#     with open("/home/beav3r/Bachelor_work/Bachelor_work/kkanji_classes_distribution_sorted.txt", "w") as file:
#         for line in lines:
#             file.write(line)

'Done'

In [57]:
"""Done"""
# Remote duplicates from the sorted file

# with open("/home/beav3r/Bachelor_work/Bachelor_work/kkanji_classes_distribution_sorted.txt", "r") as file:
#     lines = file.readlines()
#     lines = list(dict.fromkeys(lines))
#     with open("/home/beav3r/Bachelor_work/Bachelor_work/kkanji_classes_distribution_sorted.txt", "w") as file:
#         for line in lines:
#             file.write(line)

'Done'

In [58]:
# Reading svg files
tree = ET.parse("/home/beav3r/Bachelor_work/Bachelor_work/data/kanji_svg/0f9a8.svg")
root = tree.getroot()

for child in root:
    print(child.tag, child.attrib)


{http://www.w3.org/2000/svg}g {'id': 'kvg:StrokePaths_0f9a8', 'style': 'fill:none;stroke:#000000;stroke-width:3;stroke-linecap:round;stroke-linejoin:round;'}
{http://www.w3.org/2000/svg}g {'id': 'kvg:StrokeNumbers_0f9a8', 'style': 'font-size:8;fill:#808080'}


In [14]:
# GNT files exploration
"""Rewrite so it will store images in modules of their names"""
"""It is old version, new is below"""

# def read_gnt_file(file_name, output_dir):
#     try:
#         # Ensure output directory exists
#         if not os.path.exists(output_dir):
#             os.makedirs(output_dir)

#         # Open the file in read binary mode
#         with open(file_name, 'rb') as images_file:
#             # Seek to the end of the file to get its length
#             images_file.seek(0, 2)
#             file_length = images_file.tell()
#             images_file.seek(0)

#             n = 0
#             while images_file.tell() < file_length:  # Not EOF
#                 # Read number of bytes for a sample (4 bytes - long)
#                 char_length = struct.unpack('>l', images_file.read(4))[0]

#                 # Read GB code (2 bytes)
#                 char_label = images_file.read(2)

#                 # Reverse byte order from little-endian to big-endian
#                 # char_label = char_label[::-1]

#                 # Decode GB2312 into Unicode
#                 try:
#                     char_unicode = char_label.decode('gb2312')
#                 except UnicodeDecodeError:
#                     char_unicode = "unknown" + str(n)

#                 # Read character width and height (2 bytes each - short)
#                 char_wid = struct.unpack('<h', images_file.read(2))[0]
#                 char_hei = struct.unpack('<h', images_file.read(2))[0]

#                 # Read the bitmap of the grayscale image
#                 char_image_data = images_file.read(char_wid * char_hei)

#                 # Convert the raw data into an image using Pillow
#                 image = Image.frombytes('L', (char_wid, char_hei), char_image_data)

#                 # Create a valid filename using the label (sanitize the label to avoid invalid characters)
#                 safe_label = "".join([c if c.isalnum() else "_" for c in char_unicode])
#                 output_path = os.path.join(output_dir, f"{safe_label}_{n}.png")

#                 # Save the image as a PNG file
#                 image.save(output_path)

#                 n += 1

#         print(f"Total images processed: {n}")

#     except Exception as e:
#         print(f"Error occurred: {e}")

In [15]:
# gnt_file = "/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/Gnt1.0TrainPart1/001-f.gnt"
# output_dir = "/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/My_processed_Gnt1.0TrainPart1"
# read_gnt_file(gnt_file, output_dir)

Total images processed: 4024


In [75]:
# Read the first image from processed GNT file

img = cv2.imread("/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/My_processed_Gnt1.0TrainPart1/__470.png", cv2.IMREAD_GRAYSCALE)
img.shape

(69, 61)

In [43]:
# POT files exploration

def read_pot_file(file_name):
    try:
        with open(file_name, 'rb') as pot_file:
            file_open = True
            file_length = pot_file.seek(0, 2)  # Seek to end to get length
            pot_file.seek(0)  # Seek back to beginning

            n = 0
            while pot_file.tell() < file_length:  # Not EOF

                if file_length - pot_file.tell() < 2:
                    print("End of file reached1")
                    break
                # Read sample size (2 bytes - unsigned short)
                sample_size = struct.unpack('>H', pot_file.read(2))[0]

                if file_length - pot_file.tell() < 2:
                    print("End of file reached2")
                    break
                # Read tag code (4 bytes - DWORD)
                tag_code_bytes = pot_file.read(4)

                # Reverse byte order and decode GB2312 (Only last two bytes are meaningful)
                # tag_code = tag_code_bytes[:2][::-1]  # Reverse last two bytes
                tag_code = tag_code_bytes[:2]
                try:
                    char_unicode = tag_code.decode('gb2312')
                except UnicodeDecodeError:
                    char_unicode = "unknown" + str(n)

                # Sanitize char_unicode to remove non-printable characters
                char_unicode = re.sub(r'[^\w]', '_', char_unicode)  # Replace invalid characters
                char_unicode += f"_{n}"  # Add iteration number to avoid duplicates

                if file_length - pot_file.tell() < 2:
                    print("End of file reached3")
                    break
                # Read stroke number (2 bytes - unsigned short)
                stroke_num = struct.unpack('<H', pot_file.read(2))[0]

                print(f"Processing Character: {char_unicode}, Strokes: {stroke_num}, iteration: {n}")

                # Read strokes until stroke end (-1, 0) or character end (-1, -1)
                strokes = []
                for i in range(int_inf):
                    stroke = []
                    while True:
                        # Read (x, y) coordinates (2 bytes + 2 bytes), sometimes short sometimes signed short
                        if file_length - pot_file.tell() < 2:
                            print("End of file reached4")
                            break
                        x, y = struct.unpack('>hh', pot_file.read(4))

                        # Stroke end (-1, 0)
                        if x == -1 and y == 0:
                            break
                        # Character end (-1, -1)
                        elif x == -1 and y == -1:
                            break
                        else:
                            stroke.append((x, y))

                    # Exit loop if we hit character end
                    if x == -1 and y == -1:
                        break

                    strokes.append(stroke)

                # print(f"Character: {char_unicode}, Strokes: {strokes}")
                with open("/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/My_processed_Pot1.0Train/" + char_unicode + ".txt", "w") as file:
                    file.write("Strokes: " + str(strokes) + " Stroke number: " + str(len(strokes)))
                n += 1

        print(f"Total characters processed: {n}")

    except Exception as e:
        print(f"Error occurred: {e}")

In [44]:
pot_file = "/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/Pot1.0Train/001.pot"
read_pot_file(pot_file)

Processing Character: ___0, Strokes: 2, iteration: 0
Processing Character: ___1, Strokes: 2, iteration: 1
Processing Character: ___2, Strokes: 4, iteration: 2
Processing Character: ___3, Strokes: 2, iteration: 3
Processing Character: ___4, Strokes: 2, iteration: 4
Processing Character: ___5, Strokes: 1, iteration: 5
Processing Character: ___6, Strokes: 1, iteration: 6
Processing Character: ___7, Strokes: 1, iteration: 7
Processing Character: ___8, Strokes: 3, iteration: 8
Processing Character: ___9, Strokes: 2, iteration: 9
Processing Character: ___10, Strokes: 1, iteration: 10
Processing Character: ___11, Strokes: 1, iteration: 11
Processing Character: ___12, Strokes: 1, iteration: 12
Processing Character: ___13, Strokes: 1, iteration: 13
Processing Character: _0_14, Strokes: 1, iteration: 14
Processing Character: _1_15, Strokes: 1, iteration: 15
Processing Character: _2_16, Strokes: 1, iteration: 16
Processing Character: _3_17, Strokes: 1, iteration: 17
Processing Character: _4_18, S

In [2]:
# GNT files exploration
"""Rewrite so it will store images in modules of their names"""

def read_gnt_file(file_name, directories, main_output_dir):
    try:
        # Ensure output directory exists
        if not os.path.exists(main_output_dir):
            os.makedirs(main_output_dir)

        # Open the file in read binary mode
        with open(file_name, 'rb') as images_file:
            # Seek to the end of the file to get its length
            images_file.seek(0, 2)
            file_length = images_file.tell()
            images_file.seek(0)

            n = 0
            while images_file.tell() < file_length:  # Not EOF
                # Read number of bytes for a sample (4 bytes - long)
                char_length = struct.unpack('>l', images_file.read(4))[0]

                # Read GB code (2 bytes)
                char_label = images_file.read(2)

                # Decode GB2312 into Unicode
                try:
                    char_unicode = char_label.decode('gb2312')
                except UnicodeDecodeError:
                    char_unicode = "unknown" + str(n)

                # Read character width and height (2 bytes each - short)
                char_wid = struct.unpack('<h', images_file.read(2))[0]
                char_hei = struct.unpack('<h', images_file.read(2))[0]

                # Read the bitmap of the grayscale image
                char_image_data = images_file.read(char_wid * char_hei)

                # Convert the raw data into an image using Pillow
                image = Image.frombytes('L', (char_wid, char_hei), char_image_data)

                # Create a valid filename using the label (sanitize the label to avoid invalid characters)
                safe_label = "".join([c if c.isalnum() else "_" for c in char_unicode])
                
                # Add iteration number to avoid duplicates
                safe_label += f"_{n}"

                # Create a directory for the character if it doesn't exist, check if directory exists using os
                if not os.path.exists(os.path.join(main_output_dir, safe_label)):
                    os.makedirs(os.path.join(main_output_dir, safe_label))
                    directories[safe_label] = 0

                # output_path = os.path.join(output_dir, f"{safe_label}_{n}.png")
                output_path = os.path.join(main_output_dir, safe_label, f"{directories[safe_label]}.png")
                directories[safe_label] += 1

                # Save the image as a PNG file
                image.save(output_path)

                n += 1

        print(f"Total images processed: {n}")
        return directories

    except Exception as e:
        print(f"Error occurred: {e}")
        return directories

def read_gnt_folder(folder_name, directories, main_output_dir):
    try:
        for root, dirs, files in os.walk(folder_name):
            files_processed = 1
            for file in files:
                if file.endswith(".gnt"):
                    full_path = os.path.join(root, file)
                    directories = read_gnt_file(full_path, directories, main_output_dir)
                    print(f"File {files_processed} processed")
                    files_processed += 1
                
        return directories

    except Exception as e:
        print(f"Error occurred: {e}")
        return directories


In [3]:
directories = defaultdict(int)
main_output_dir = "/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/My_processed_Gnt1.0Train"

# clear main_output_dir dirs if they exist
for root, dirs, files in os.walk(main_output_dir):
    for dir in dirs:
        # Force delete the directory
        os.system("rm -rf " + os.path.join(root, dir))

folder_name = "/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/Gnt1.0TrainPart1"
directories = read_gnt_folder(folder_name, directories, main_output_dir)
print("Folder 1 done")

# Save directories to txt file
with open("/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/directories.txt", "w") as file:
    for key, value in directories.items():
        file.write(key + " " + str(value) + "\n")


Total images processed: 4029
File 1 processed
Total images processed: 3971
File 2 processed
Total images processed: 4021
File 3 processed
Total images processed: 3760
File 4 processed
Total images processed: 4027
File 5 processed
Total images processed: 4029
File 6 processed
Total images processed: 4011
File 7 processed
Total images processed: 4035
File 8 processed
Total images processed: 4036
File 9 processed
Total images processed: 4033
File 10 processed
Total images processed: 4036
File 11 processed
Total images processed: 4027
File 12 processed
Total images processed: 3975
File 13 processed
Total images processed: 4031
File 14 processed
Total images processed: 4023
File 15 processed
Total images processed: 4033
File 16 processed
Total images processed: 4022
File 17 processed
Total images processed: 4022
File 18 processed
Total images processed: 4026
File 19 processed
Total images processed: 4031
File 20 processed
Total images processed: 4033
File 21 processed
Total images processed

In [4]:
# Read directories from txt file
directories = defaultdict(int)
with open("/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/directories.txt", "r") as file:
    lines = file.readlines()
    for line in lines:
        key, value = line.split()
        directories[key] = int(value)

folder_name = "/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/Gnt1.0TrainPart2"
directories = read_gnt_folder(folder_name, directories, main_output_dir)
print("Folder 2 done")

# Save directories to txt file
with open("/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/directories.txt", "w") as file:
    for key, value in directories.items():
        file.write(key + " " + str(value) + "\n")

Total images processed: 4014
File 1 processed
Total images processed: 3980
File 2 processed
Total images processed: 4036
File 3 processed
Total images processed: 4028
File 4 processed
Total images processed: 4028
File 5 processed
Total images processed: 4028
File 6 processed
Total images processed: 4030
File 7 processed
Total images processed: 4010
File 8 processed
Total images processed: 4024
File 9 processed
Total images processed: 4014
File 10 processed
Total images processed: 4013
File 11 processed
Total images processed: 4033
File 12 processed
Total images processed: 4006
File 13 processed
Total images processed: 4033
File 14 processed
Total images processed: 4024
File 15 processed
Total images processed: 4016
File 16 processed
Total images processed: 4025
File 17 processed
Total images processed: 4034
File 18 processed
Total images processed: 4002
File 19 processed
Total images processed: 4037
File 20 processed
Total images processed: 3988
File 21 processed
Total images processed

In [5]:
# Read directories from txt file
directories = defaultdict(int)
with open("/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/directories.txt", "r") as file:
    lines = file.readlines()
    for line in lines:
        key, value = line.split()
        directories[key] = int(value)

folder_name = "/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/Gnt1.0TrainPart3"
directories = read_gnt_folder(folder_name, directories, main_output_dir)
print("Folder 3 done")

# Save directories to txt file
with open("/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/directories.txt", "w") as file:
    for key, value in directories.items():
        file.write(key + " " + str(value) + "\n")

Total images processed: 4031
File 1 processed
Total images processed: 3983
File 2 processed
Total images processed: 4035
File 3 processed
Total images processed: 3736
File 4 processed
Total images processed: 4015
File 5 processed
Total images processed: 3965
File 6 processed
Total images processed: 4033
File 7 processed
Total images processed: 4034
File 8 processed
Total images processed: 4034
File 9 processed
Total images processed: 4007
File 10 processed
Total images processed: 4030
File 11 processed
Total images processed: 4032
File 12 processed
Total images processed: 4017
File 13 processed
Total images processed: 4027
File 14 processed
Total images processed: 4034
File 15 processed
Total images processed: 4021
File 16 processed
Total images processed: 3760
File 17 processed
Total images processed: 4029
File 18 processed
Total images processed: 4033
File 19 processed
Total images processed: 4036
File 20 processed
Total images processed: 4030
File 21 processed
Total images processed

In [6]:
# Read directories from txt file
directories = defaultdict(int)
with open("/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/directories.txt", "r") as file:
    lines = file.readlines()
    for line in lines:
        key, value = line.split()
        directories[key] = int(value)

main_output_dir = "/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/My_processed_Gnt1.1Train"

# clear main_output_dir dirs if they exist
for root, dirs, files in os.walk(main_output_dir):
    for dir in dirs:
        # Force delete the directory
        os.system("rm -rf " + os.path.join(root, dir))

folder_name = "/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/Gnt1.1TrainPart1"
directories = read_gnt_folder(folder_name, directories, main_output_dir)
print("Folder 4 done")

# Save directories to txt file
with open("/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/directories.txt", "w") as file:
    for key, value in directories.items():
        file.write(key + " " + str(value) + "\n")

Total images processed: 3926
File 1 processed
Total images processed: 3919
File 2 processed
Total images processed: 3919
File 3 processed
Total images processed: 3894
File 4 processed
Total images processed: 3926
File 5 processed
Total images processed: 3907
File 6 processed
Total images processed: 3915
File 7 processed
Total images processed: 3922
File 8 processed
Total images processed: 3919
File 9 processed
Total images processed: 3918
File 10 processed
Total images processed: 3897
File 11 processed
Total images processed: 3923
File 12 processed
Total images processed: 3925
File 13 processed
Total images processed: 3922
File 14 processed
Total images processed: 3923
File 15 processed
Total images processed: 3892
File 16 processed
Total images processed: 3922
File 17 processed
Total images processed: 3904
File 18 processed
Total images processed: 3891
File 19 processed
Total images processed: 3922
File 20 processed
Total images processed: 3905
File 21 processed
Total images processed

In [7]:
# Read directories from txt file
directories = defaultdict(int)
with open("/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/directories.txt", "r") as file:
    lines = file.readlines()
    for line in lines:
        key, value = line.split()
        directories[key] = int(value)

folder_name = "/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/Gnt1.1TrainPart2"
directories = read_gnt_folder(folder_name, directories, main_output_dir)
print("Folder 5 done")

# Save directories to txt file
with open("/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/directories.txt", "w") as file:
    for key, value in directories.items():
        file.write(key + " " + str(value) + "\n")

Total images processed: 3915
File 1 processed
Total images processed: 3893
File 2 processed
Total images processed: 3904
File 3 processed
Total images processed: 3919
File 4 processed
Total images processed: 3923
File 5 processed
Total images processed: 3910
File 6 processed
Total images processed: 3894
File 7 processed
Total images processed: 3925
File 8 processed
Total images processed: 3913
File 9 processed
Total images processed: 3873
File 10 processed
Total images processed: 3926
File 11 processed
Total images processed: 3925
File 12 processed
Total images processed: 3872
File 13 processed
Total images processed: 3907
File 14 processed
Total images processed: 3922
File 15 processed
Total images processed: 3908
File 16 processed
Total images processed: 3914
File 17 processed
Total images processed: 3915
File 18 processed
Total images processed: 3919
File 19 processed
Total images processed: 3895
File 20 processed
Total images processed: 3893
File 21 processed
Total images processed

In [8]:
# Read directories from txt file
directories = defaultdict(int)
with open("/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/directories.txt", "r") as file:
    lines = file.readlines()
    for line in lines:
        key, value = line.split()
        directories[key] = int(value)

main_output_dir = "/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/My_processed_Gnt1.2Train"

# clear main_output_dir dirs if they exist
for root, dirs, files in os.walk(main_output_dir):
    for dir in dirs:
        # Force delete the directory
        os.system("rm -rf " + os.path.join(root, dir))

folder_name = "/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/Gnt1.2TrainPart1"
directories = read_gnt_folder(folder_name, directories, main_output_dir)
print("Folder 6 done")

# Save directories to txt file
with open("/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/directories.txt", "w") as file:
    for key, value in directories.items():
        file.write(key + " " + str(value) + "\n")

Total images processed: 3487
File 1 processed
Total images processed: 3484
File 2 processed
Total images processed: 3484
File 3 processed
Total images processed: 3485
File 4 processed
Total images processed: 3474
File 5 processed
Total images processed: 3452
File 6 processed
Total images processed: 3485
File 7 processed
Total images processed: 3485
File 8 processed
Total images processed: 3482
File 9 processed
Total images processed: 3465
File 10 processed
Total images processed: 3476
File 11 processed
Total images processed: 3477
File 12 processed
Total images processed: 3484
File 13 processed
Total images processed: 3444
File 14 processed
Total images processed: 3464
File 15 processed
Total images processed: 3486
File 16 processed
Total images processed: 3476
File 17 processed
Total images processed: 3483
File 18 processed
Total images processed: 3482
File 19 processed
Total images processed: 3482
File 20 processed
Total images processed: 3475
File 21 processed
Total images processed

In [9]:
# Read directories from txt file
directories = defaultdict(int)
with open("/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/directories.txt", "r") as file:
    lines = file.readlines()
    for line in lines:
        key, value = line.split()
        directories[key] = int(value)

folder_name = "/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/Gnt1.2TrainPart2"
directories = read_gnt_folder(folder_name, directories, main_output_dir)
print("Folder 7 done")

# Save directories to txt file
with open("/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/directories.txt", "w") as file:
    for key, value in directories.items():
        file.write(key + " " + str(value) + "\n")

Total images processed: 3479
File 1 processed
Total images processed: 3486
File 2 processed
Total images processed: 3488
File 3 processed
Total images processed: 3475
File 4 processed
Total images processed: 3475
File 5 processed
Total images processed: 3474
File 6 processed
Total images processed: 3484
File 7 processed
Total images processed: 3474
File 8 processed
Total images processed: 3487
File 9 processed
Total images processed: 3477
File 10 processed
Total images processed: 3477
File 11 processed
Total images processed: 3478
File 12 processed
Total images processed: 3486
File 13 processed
Total images processed: 3471
File 14 processed
Total images processed: 3488
File 15 processed
Total images processed: 3487
File 16 processed
Total images processed: 3467
File 17 processed
Total images processed: 3468
File 18 processed
Total images processed: 3474
File 19 processed
Total images processed: 3423
File 20 processed
Total images processed: 3480
File 21 processed
Total images processed

In [10]:
# Read directories from txt file
directories = defaultdict(int)
with open("/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/directories.txt", "r") as file:
    lines = file.readlines()
    for line in lines:
        key, value = line.split()
        directories[key] = int(value)

main_output_dir = "/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/My_processed_Gnt1.0Test"

# clear main_output_dir dirs if they exist
for root, dirs, files in os.walk(main_output_dir):
    for dir in dirs:
        # Force delete the directory
        os.system("rm -rf " + os.path.join(root, dir))

folder_name = "/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/Gnt1.0Test"
directories = read_gnt_folder(folder_name, directories, main_output_dir)
print("Folder 8 done")

# Save directories to txt file
with open("/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/directories.txt", "w") as file:
    for key, value in directories.items():
        file.write(key + " " + str(value) + "\n")

Total images processed: 4034
File 1 processed
Total images processed: 3997
File 2 processed
Total images processed: 4035
File 3 processed
Total images processed: 4021
File 4 processed
Total images processed: 4030
File 5 processed
Total images processed: 4024
File 6 processed
Total images processed: 4029
File 7 processed
Total images processed: 4018
File 8 processed
Total images processed: 4014
File 9 processed
Total images processed: 4020
File 10 processed
Total images processed: 4028
File 11 processed
Total images processed: 4017
File 12 processed
Total images processed: 4030
File 13 processed
Total images processed: 4027
File 14 processed
Total images processed: 4013
File 15 processed
Total images processed: 4024
File 16 processed
Total images processed: 4018
File 17 processed
Total images processed: 4014
File 18 processed
Total images processed: 4036
File 19 processed
Total images processed: 4034
File 20 processed
Total images processed: 4027
File 21 processed
Total images processed

In [11]:
# Read directories from txt file
directories = defaultdict(int)
with open("/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/directories.txt", "r") as file:
    lines = file.readlines()
    for line in lines:
        key, value = line.split()
        directories[key] = int(value)

main_output_dir = "/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/My_processed_Gnt1.1Test"

# clear main_output_dir dirs if they exist
for root, dirs, files in os.walk(main_output_dir):
    for dir in dirs:
        # Force delete the directory
        os.system("rm -rf " + os.path.join(root, dir))

folder_name = "/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/Gnt1.1Test"
directories = read_gnt_folder(folder_name, directories, main_output_dir)
print("Folder 9 done")

# Save directories to txt file
with open("/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/directories.txt", "w") as file:
    for key, value in directories.items():
        file.write(key + " " + str(value) + "\n")

Total images processed: 3906
File 1 processed
Total images processed: 3925
File 2 processed
Total images processed: 3922
File 3 processed
Total images processed: 3666
File 4 processed
Total images processed: 3902
File 5 processed
Total images processed: 3917
File 6 processed
Total images processed: 3890
File 7 processed
Total images processed: 3923
File 8 processed
Total images processed: 3917
File 9 processed
Total images processed: 3899
File 10 processed
Total images processed: 3920
File 11 processed
Total images processed: 3918
File 12 processed
Total images processed: 3863
File 13 processed
Total images processed: 3911
File 14 processed
Total images processed: 3913
File 15 processed
Total images processed: 3922
File 16 processed
Total images processed: 3917
File 17 processed
Total images processed: 3914
File 18 processed
Total images processed: 3914
File 19 processed
Total images processed: 3923
File 20 processed
Total images processed: 3926
File 21 processed
Total images processed

In [12]:
# Read directories from txt file
directories = defaultdict(int)
with open("/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/directories.txt", "r") as file:
    lines = file.readlines()
    for line in lines:
        key, value = line.split()
        directories[key] = int(value)

main_output_dir = "/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/My_processed_Gnt1.2Test"

# clear main_output_dir dirs if they exist
for root, dirs, files in os.walk(main_output_dir):
    for dir in dirs:
        # Force delete the directory
        os.system("rm -rf " + os.path.join(root, dir))

folder_name = "/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/Gnt1.2Test"
directories = read_gnt_folder(folder_name, directories, main_output_dir)
print("Folder 10 done")

# Save directories to txt file
with open("/home/beav3r/Bachelor_work/Bachelor_work/data/CASIA Online and Offline Chinese Handwriting Databases/directories.txt", "w") as file:
    for key, value in directories.items():
        file.write(key + " " + str(value) + "\n")

Total images processed: 3474
File 1 processed
Total images processed: 3483
File 2 processed
Total images processed: 3476
File 3 processed
Total images processed: 3485
File 4 processed
Total images processed: 3489
File 5 processed
Total images processed: 3456
File 6 processed
Total images processed: 3487
File 7 processed
Total images processed: 3488
File 8 processed
Total images processed: 3484
File 9 processed
Total images processed: 3476
File 10 processed
Total images processed: 3489
File 11 processed
Total images processed: 3471
File 12 processed
Total images processed: 3480
File 13 processed
Total images processed: 3485
File 14 processed
Total images processed: 3444
File 15 processed
Total images processed: 3486
File 16 processed
Total images processed: 3472
File 17 processed
Total images processed: 3478
File 18 processed
Total images processed: 3490
File 19 processed
Total images processed: 3489
File 20 processed
Total images processed: 3490
File 21 processed
Total images processed